# **NOTE**

ONLY NEEDS TO RUN ONCE! Make sure you absolutely need to run this before running it. If datafiles already exist, then you probably don't need to rerun this.

Do NOT run without unzipping `T_ONTIME_REPORTING_unprocessedData.zip` file first and move data files to main data directory, otherwise this code will fail.

# Set up environment

In [12]:
import os
import pandas as pd
import numpy as np
import pdb
import csv
import glob

from sklearn.preprocessing import OrdinalEncoder

# Constants

In [13]:
datadir='/Users/madelinefrank/Documents/KSU/Course_Work/5_Summer2023/CS_7265/Project/data'
yrs=['2017','2018','2019']
mos=['01','02','03','04','05','06','07','08','09','10','11','12']
fn_fmt='T_ONTIME_REPORTING_{yr}{mo}.csv'

cols=['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','FL_DATE','OP_UNIQUE_CARRIER',
      'ORIGIN','ORIGIN_STATE_ABR',
      'DEST',    'DEST_STATE_ABR',
      'CRS_DEP_TIME','DEP_TIME','DEP_DELAY','DEP_DELAY_NEW','DEP_DEL15','DEP_DELAY_GROUP',
      'CRS_ARR_TIME','ARR_TIME','ARR_DELAY','ARR_DELAY_NEW','ARR_DEL15','ARR_DELAY_GROUP',
      'CANCELLED','DIVERTED','CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME',
      'DISTANCE','DISTANCE_GROUP',
      'CARRIER_DELAY', 'WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']
renamecols = {'YEAR':'YR','MONTH':'MON','DAY_OF_MONTH':'DAYOFMONTH','DAY_OF_WEEK':'DAYOFWEEK',
            'OP_UNIQUE_CARRIER':'CARRIER',
           'ORIGIN':'ORIG','ORIGIN_STATE_ABR':'ORIG_ST','DEST_STATE_ABR':'DEST_ST',
           'CRS_DEP_TIME':'CRS_T_DEP','DEP_TIME':'ACT_T_DEP',
           'CRS_ARR_TIME':'CRS_T_ARR','ARR_TIME':'ACT_T_ARR',
           'DEP_DELAY_NEW':'DEP_DELAY0','DEP_DEL15':'DEP_DEL15_TF','DEP_DELAY_GROUP':'DEP_DEL_GRP',
           'ARR_DELAY_NEW':'ARR_DELAY0','ARR_DEL15':'ARR_DEL15_TF','ARR_DELAY_GROUP':'ARR_DEL_GRP',
           'CRS_ELAPSED_TIME':'CRS_T_ELAPSED','ACTUAL_ELAPSED_TIME':'ACT_T_ELAPSED',
           'DISTANCE':'DIST','DISTANCE_GROUP':'DIST_GRP',
           'WEATHER_DELAY':'WX_DELAY','SECURITY_DELAY':'SEC_DELAY'}
airports=['ATL', 'ORD', 'DEN', 'LAX', 'DFW', 'PHX', 'SFO', 'LAS', 'SEA', 'MSP',
          'MCO', 'DTW', 'BOS', 'SLC', 'CLT', 'BWI', 'EWR', 'JFK', 'FLL', 'MDW',
          'IAH', 'SAN', 'LGA', 'PHL', 'TPA', 'DCA', 'MIA', 'DAL', 'PDX', 'BNA',
          'STL', 'HOU', 'AUS', 'OAK', 'SJC', 'MSY', 'SMF', 'MCI', 'SNA', 'IAD',
          'RDU', 'SAT', 'RSW', 'MKE', 'PIT', 'CLE', 'IND', 'BUR', 'PBI', 'SJU',
          'CMH', 'BDL', 'ONT', 'ABQ', 'CVG', 'OMA', 'ANC', 'JAX', 'BUF', 'LGB',
          'BOI', 'RNO', 'TUS', 'OKC', 'HNL', 'MEM', 'PVD', 'CHS', 'GEG', 'TUL',
          'RIC', 'ELP', 'ORF', 'ALB', 'GRR', 'COS', 'SDF', 'PSP', 'OGG', 'BHM',
          'FAT', 'MSN', 'DSM', 'ICT', 'ROC', 'MYR', 'SAV', 'LIT', 'SYR', 'MHT',
          'ISP', 'DAY', 'ASE', 'CID', 'GSP']
oconus=['BET','BRW','CDV','DLG','ADQ','FAI','GST','JNU','AKN','KTN','ANC','OME','OTZ','PSG','SCC','SIT','WRG','YAK','ADK','PPG','GUM','KOA','LIH','HNL','OGG','ITO']
airlines=['AA','B6','DL','F9','NK','UA','WN']
cols_model = ['DAYOFWEEK','DAYOFYEAR','CARRIER','ORIG','ORIG_ST','DEST',
              'DEST_ST','CRS_HR_DEP','CRS_HR_ARR','DIST_GRP','ARR_DEL15_TF']

#Settings
# Determine encoding method for categorical data
#isOneHot = True #For  True = use 1 hot encoding, False = use regular numeric encoding
cols2enc=['CARRIER','ORIG','ORIG_ST','DEST','DEST_ST']
enc_groups={'CARRIER':['CARRIER'], 'AIRPORT': ['ORIG','DEST'], 'STATE': ['ORIG_ST','DEST_ST']}

# Helper Functions

In [14]:
# Get total number of files to be preprocessed
def getNumFNs():
    num_fns = 0
    for yr in yrs:
        fns = glob.glob(os.path.join(datadir,fn_fmt.format(yr=yr,mo='*')))
        num_fns+=len(fns)
    return num_fns

In [15]:
# Clean and reformat data (no encoding)
def fmt_clean_data(df):
    try:
        # airports=pd.read_csv(os.path.join(datadir,'airports_delay_data_xs.csv'))
        df.FL_DATE=pd.to_datetime(df.FL_DATE,utc=False,dayfirst=False,yearfirst=False,format='%m/%d/%y %H:%M')
        tmp=['DIVERTED','CANCELLED','DEP_DEL15','ARR_DEL15']
        df[tmp]=df[tmp].astype('bool')
        # Remove diverted or cancelled flights
        df=df.loc[df['CANCELLED']==False]
        df=df.loc[df['DIVERTED'] ==False]
        df.drop(columns=['DIVERTED','CANCELLED'],inplace=True)
        df.rename(columns=renamecols,inplace=True)
        df=df.loc[df['CARRIER'].isin(airlines)]
        df=df.loc[~df['ORIG'].isin(oconus) & ~df['DEST'].isin(oconus)]
        df=df.loc[df['ORIG'].isin(airports) & df['DEST'].isin(airports)]
        df.insert(4,'DAYOFYEAR',df.FL_DATE.apply(lambda x: x.dayofyear))
        df.drop(columns=['FL_DATE'],inplace=True)
        df.dropna(how='any',subset=['ACT_T_ARR'],inplace=True)
        df['CRS_T_DEP']=df['CRS_T_DEP'].apply(lambda x: '{0:0>4}'.format(x))
        df['ACT_T_DEP']=df['ACT_T_DEP'].astype('int').apply(lambda x: '{0:0>4}'.format(x))
        df['CRS_T_ARR']=df['CRS_T_ARR'].apply(lambda x: '{0:0>4}'.format(x))
        df['ACT_T_ARR']=df['ACT_T_ARR'].astype('int').apply(lambda x: '{0:0>4}'.format(x))
        df.insert(11,'CRS_HR_DEP',df.CRS_T_DEP.apply(lambda x: x[:2]))
        df.insert(18,'CRS_HR_ARR',df.CRS_T_ARR.apply(lambda x: x[:2]))
        df[['CRS_HR_DEP','CRS_HR_ARR']]=df[['CRS_HR_DEP','CRS_HR_ARR']].astype('int')

        df.reset_index(drop=True,inplace=True)
        df=df[cols_model]

    except:
        # df.dropna(how='any',subset=['ACT_T_ARR'],inplace=True)
        pdb.set_trace()
    return df

In [16]:
# Get filename for model data
def getModelFN(fn,yr):
    tmp = os.path.basename(fn).split(yr)
    tmp = tmp[0]+'Model'+yr+tmp[1]
    # Remove file if it already exists
    if os.path.exists(os.path.join(datadir,tmp)):
        os.remove(os.path.join(datadir,tmp))
    return tmp

In [ ]:
# Create and save dictionaries for categorical feature ordinal encoding
def encode_cat_data(dat, grp_name):
    oe = OrdinalEncoder()
    #oe_nums=oe.fit_transform(dat['ORIG'].array.reshape((-1,1)))
    oe_nums=oe.fit_transform(np.asarray(dat).reshape((-1,1)))
    oe_lbls=oe.inverse_transform(oe_nums)
    oenc2lbls=dict(zip(oe_nums.squeeze(),oe_lbls.squeeze()))
    # lbls2oenc=dict(zip(oe_lbls.squeeze(),oe_nums.squeeze()))
    w = csv.writer(open(os.path.join(datadir,'oenc2lbls_'+grp_name+'.csv'),'w')) # save for future use
    for oenc, lbl in oenc2lbls.items():
        w.writerow([oenc,lbl])
    # x = csv.writer(open(os.path.join(datadir,'lbls2oenc_'+grp_name+'.csv'),'w')) # save for future use
    # for lbl1, oenc1 in lbls2oenc.items():
    #     x.writerow([lbl1,oenc1])

In [17]:
# Group together specified categorical features (see "Constants" section above)
# to get all possible values
def groupEncFeats(enc_dict):
    for grp_name,grp in enc_groups.items():
        for g in grp:
            a=enc_dict.pop(g)
            if (grp_name in enc_groups):
                enc_dict.update({grp_name: []})
            b=enc_dict.pop(grp_name)
            a.extend(b)
            enc_dict.update({grp_name: pd.unique(a)})
        encode_cat_data(enc_dict[grp_name],grp_name)

In [ ]:
# TEST
# def mk_model_data(df):
#     # Init label encoder for categorical data
#     oe = OrdinalEncoder()
#     oe_nums=oe.fit_transform(df['ORIG'].array.reshape((-1,1)))
#     oe_lbls=oe.inverse_transform(oe_nums)
#     print(df.head)
#     df['ORIG']=pd.Series(oe_nums.squeeze(),name='ORIG')
#     print(df.head)

#     lbls_orig_arpt = oe.fit_transform(df['ORIG'])
# blah = np.reshape(['ATL','ORD','LAX','ATL','MCO','JFK','JFK','09B','DEN'],(-1, 1))
# oe = OrdinalEncoder(dtype=np.int64)
# oe_nums=oe.fit_transform(blah)
# oe_lbls=oe.inverse_transform(oe_nums)

# Main

In [19]:
# Get total number of files that will be preprocessed
num_fns=getNumFNs()

# Initiate encoding dictionary (will be saved for future use)
enc_dict=dict()

count=0
for yr in yrs:
    fns = glob.glob(os.path.join(datadir,fn_fmt.format(yr=yr,mo='*')))
    for fn in fns:
        DF=pd.read_csv(fn,usecols=cols,index_col=False)
        DF=fmt_clean_data(df=DF)
        # Write cleaned data to new file
        fn_new=getModelFN(fn,yr)
        DF.to_csv(os.path.join(datadir,fn_new),header=True,index=False)
        # Get all possible values for categorical features (see "Constants" section above)
        for col in cols2enc:
            if (col not in enc_dict.keys()):
                enc_dict.setdefault(col,[])
            enc_dict[col].extend(pd.unique(DF[col]))
            enc_dict[col]=list(set(enc_dict[col]))
        count+=1
        print('{c:>3d}/{n}'.format(c=count,n=num_fns))


groupEncFeats(enc_dict)
    

# Group together specified categorical features (see "Constants" section above)
# for grp_name,grp in enc_groups.items():
#     for g in grp:
#         a=enc_dict.pop(g)
#         if grp_name in enc_groups:
#             enc_dict.update({grp_name: []})
#         b=enc_dict.pop(grp_name)
#         a.extend(b)
#         enc_dict.update({grp_name: pd.unique(a)})

  1/36
  2/36
  3/36
  4/36
  5/36
  6/36
  7/36
  8/36
  9/36
 10/36
 11/36
 12/36
 13/36
 14/36
 15/36
 16/36
 17/36
 18/36
 19/36
 20/36
 21/36
 22/36
 23/36
 24/36
 25/36
 26/36
 27/36
 28/36
 29/36
 30/36
 31/36
 32/36
 33/36
 34/36
 35/36
 36/36
